In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

# %cd "/content/drive/My Drive/"
# !git clone https://github.com/jjonhwa/HKNU-Lecture.git

In [1]:
!pip3 install -qq streamlit langchain langchain-community pdfminer.six faiss-cpu google-search-results
!npm install localtunnel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 4s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [2]:
import warnings
warnings.filterwarnings("ignore")

import streamlit as st

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import ChatMessage

# Function to extract text from an PDF file
from pdfminer.high_level import extract_text

In [3]:
# @title set API key
import os
import getpass

# Get the Upstage API key using getpass
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI API key: ")

print("API key has been set successfully.")

KeyboardInterrupt: Interrupted by user

### NOTEBOOK 실습

In [ ]:
from google.colab import files

# 파일 업로드 창이 뜨고, 선택한 파일이 Colab 환경에 업로드됨
uploaded = files.upload()

# 업로드된 파일 확인
for filename in uploaded.keys():
    print(f'파일 이름: {filename}')

Saving 주택임대차보호법(법률)(제19356호)(20230719) (3).pdf to 주택임대차보호법(법률)(제19356호)(20230719) (3).pdf
파일 이름: 주택임대차보호법(법률)(제19356호)(20230719) (3).pdf


In [ ]:
# loader
def get_pdf_text(filename):
    raw_text = extract_text(filename)
    return raw_text

# 문서 로드 및 청킹 (Loader + Splitter)
def process_uploaded_file(FILE_PATH):
    # Load document if file is uploaded
    if FILE_PATH is not None:

        # loader
        raw_text = get_pdf_text(FILE_PATH)

        # splitter
        text_splitter = CharacterTextSplitter(
            separator = "\n\n",     # TODO: 어떤 문구를 기준으로? (스페이스바, 엔터, ',' 등)
            chunk_size = 1000,      # TODO: 문서의 잘림 크기는 몇으로?
            chunk_overlap = 200,    # TODO: 겹치는 길이는 몇으로?
        )
        all_splits = text_splitter.create_documents([raw_text])
        print("총 " + str(len(all_splits)) + "개의 passage")

        # storage
        vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

        return vectorstore, raw_text
    return None

총 24개의 passage


In [ ]:
FILE_PATH = "/content/주택임대차보호법(법률)(제19356호)(20230719) (3).pdf"
vectorstore, raw_text = process_uploaded_file(FILE_PATH)

In [ ]:
# handle streaming conversation
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# RAG 기반으로 답변 생성 (검색(Retriever), 생성(Generator), 연결(Chaining))
def generate_response(query_text, vectorstore, callback):

    # retriever
    docs_list = vectorstore.similarity_search(query_text, k=3) # TODO: 연관성 있는 문서는 몇 개?!
    docs = ""
    for i, doc in enumerate(docs_list):
        docs += f"'문서{i+1}':{doc.page_content}\n"

    # generator
    llm = ChatOpenAI(
        model_name="gpt-4o-mini",
        temperature=0,
        streaming=True,
        # callbacks=[callback]
    )

    # chaining
    # prompt formatting
#     system_prompt = """
#     너는 문서에 대해 질의응답을 하는 조교야. \n
# 주어진 문서를 참고하여 사용자의 질문에 답변을 해줘. \n
# 문서에 내용이 정확하게 나와있지 않으면 대답하지 마.

# 너는 괴팍한 성격을 가지고 있고 완전 츤데래야."""

    rag_prompt = [
        SystemMessage(
            # content=system_prompt,
            content="너는 문서에 대해 질의응답을 하는 '조교'야. 주어진 문서를 참고하여 사용자의 질문에 답변을 해줘. 문서에 내용이 정확하게 나와있지 않으면 대답하지 마. 이모티콘을 사용해서 친근하게 답변해줘!"
        ),
        HumanMessage(
            content=f"질문:{query_text}\n\n{docs}"
        ),
    ]

    response = llm(rag_prompt)
    return response.content

def generate_summarize(raw_text, callback):

    # generator
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, streaming=True, callbacks=[callback])

    # prompt formatting
    rag_prompt = [
        SystemMessage(
            content="다음 나올 문서를 'Notion style'로 요약해줘. 중요한 내용만."
        ),
        HumanMessage(
            content=raw_text
        ),
    ]

    response = llm(rag_prompt)
    return response.content

In [ ]:
from IPython.display import clear_output

class JupyterContainer:
    def markdown(self, text):
        clear_output(wait=True)

prompt = "주택임대차 보호법에 대하여 설명해줘"
stream_handler = StreamHandler(JupyterContainer())
response = generate_response(prompt, vectorstore, stream_handler)

### Streamlit 실습

In [ ]:
%%writefile requirements.txt

streamlit==1.46.1
langchain==0.3.26
langchain-community==0.3.27
pdfminer.six==20250506
faiss-cpu==1.11.0
httpx==0.28.1
openai==1.93.0

In [4]:
%%writefile .env

OPENAI_API_KEY="{}"

Writing .env


In [5]:
%%writefile app.py

import streamlit as st
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import ChatMessage

# Function to extract text from an PDF file
from pdfminer.high_level import extract_text

from dotenv import load_dotenv
load_dotenv()

# loader
def get_pdf_text(filename):
    raw_text = extract_text(filename)
    return raw_text

# 문서 로드 및 청킹 (Loader + Splitter)
def process_uploaded_file(FILE_PATH):
    # Load document if file is uploaded
    if FILE_PATH is not None:

        # loader
        raw_text = get_pdf_text(FILE_PATH)

        # splitter
        text_splitter = CharacterTextSplitter(
            separator = "\n\n",     # TODO: 어떤 문구를 기준으로? (스페이스바, 엔터, ',' 등)
            chunk_size = 1000,      # TODO: 문서의 잘림 크기는 몇으로?
            chunk_overlap = 200,    # TODO: 겹치는 길이는 몇으로?
        )
        all_splits = text_splitter.create_documents([raw_text])
        print("총 " + str(len(all_splits)) + "개의 passage")

        # storage
        vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

        return vectorstore, raw_text
    return None

# handle streaming conversation
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# RAG 기반으로 답변 생성 (검색(Retriever), 생성(Generator), 연결(Chaining))
def generate_response(query_text, vectorstore, callback):

    # retriever
    docs_list = vectorstore.similarity_search(query_text, k=3) # TODO: 연관성 있는 문서는 몇 개?!
    docs = ""
    for i, doc in enumerate(docs_list):
        docs += f"'문서{i+1}':{doc.page_content}\n"

    # generator
    llm = ChatOpenAI(
        model_name="gpt-4o-mini",
        temperature=0,
        streaming=True,
        callbacks=[callback]
    )

    # chaining
    # prompt formatting
#     system_prompt = """
#     너는 문서에 대해 질의응답을 하는 조교야. \n
# 주어진 문서를 참고하여 사용자의 질문에 답변을 해줘. \n
# 문서에 내용이 정확하게 나와있지 않으면 대답하지 마.

# 너는 괴팍한 성격을 가지고 있고 완전 츤데래야."""

    rag_prompt = [
        SystemMessage(
            # content=system_prompt,
            content="너는 문서에 대해 질의응답을 하는 '조교'야. 주어진 문서를 참고하여 사용자의 질문에 답변을 해줘. 문서에 내용이 정확하게 나와있지 않으면 대답하지 마. 이모티콘을 사용해서 친근하게 답변해줘!"
        ),
        HumanMessage(
            content=f"질문:{query_text}\n\n{docs}"
        ),
    ]

    response = llm(rag_prompt)
    return response.content

def generate_summarize(raw_text, callback):

    # generator
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, streaming=True, callbacks=[callback])

    # prompt formatting
    rag_prompt = [
        SystemMessage(
            content="다음 나올 문서를 'Notion style'로 요약해줘. 중요한 내용만."
        ),
        HumanMessage(
            content=raw_text
        ),
    ]

    response = llm(rag_prompt)
    return response.content

# page title
st.set_page_config(page_title='🦜🔗 문서 기반 요약 및 QA 챗봇')
st.title('🦜🔗 문서 기반 요약 및 QA 챗봇')

# file upload --> PDF만 받을 수 있도록 한다.
# uploaded_file = st.file_uploader('Upload an document', type=['hwp','pdf'])
st.markdown("""
	<style>
	.main { background-color: #f5f5f5; }
	.sidebar .sidebar-content { background-color: #f0f0f0; }
	.stButton>button { background-color: #4CAF50; color: white; }
	</style>
	""", unsafe_allow_html=True)

# 파일 업로드
st.sidebar.header('📄 파일 업로드')
uploaded_file = st.sidebar.file_uploader('문서를 업로드하세요', type=['pdf'])

# file upload logic
if uploaded_file:
    vectorstore, raw_text = process_uploaded_file(uploaded_file)

    # 초기에 한 번 만들고 session_state에 보관 --> session_state에 접근해서 반복하여 활용
    if vectorstore:
        st.session_state['vectorstore'] = vectorstore
        st.session_state['raw_text'] = raw_text

# chatbot greatings
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        ChatMessage(
            role="assistant", content="안녕하세요! 저는 문서에 대한 이해를 도와주는 챗봇입니다. 어떤게 궁금하신가요?"
        )
    ]

# conversation history print --> 주석 처리하고 비교해보기
for msg in st.session_state.messages:
    st.chat_message(msg.role).write(msg.content)

# message interaction
if prompt := st.chat_input("'요약'이라고 입력해보세요!"):
    st.session_state.messages.append(ChatMessage(role="user", content=prompt))
    st.chat_message("user").write(prompt)

    with st.chat_message("assistant"):
        stream_handler = StreamHandler(st.empty())

        if prompt == "요약":
            response = generate_summarize(st.session_state['raw_text'],stream_handler)

        else:
            response = generate_response(prompt, st.session_state['vectorstore'], stream_handler)

    st.session_state.messages.append(
        ChatMessage(role="assistant", content=response)
    )

Writing app.py


In [6]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.196.181.23
⠙⠹your url is: https://flat-doors-smile.loca.lt
